### Praktikum 4: Membuat Objek dari Data

In [ ]:
import pandas as pd
from abc import ABC, abstractmethod

# --- Definisi Kelas (Salin dari Praktikum 3) ---
class Lokasi(ABC):
    def __init__(self, nama: str, latitude: float, longitude: float):
        self.nama = str(nama) if nama else "Tanpa Nama"
        try:
            self.latitude = float(latitude)
            self.longitude = float(longitude)
        except (ValueError, TypeError, SystemError):
            # print(f" -> Peringatan: Koordinat tidak valid untuk '{self.nama}'. Set ke (0.0, 0.0).")
            self.latitude = 0.0
            self.longitude = 0.0

    def get_koordinat(self) -> tuple:
        return (self.latitude, self.longitude)

    @abstractmethod
    def get_info_popup(self) -> str:
        pass

    def __repr__(self) -> str:
        return f"{type(self).__name__}(nama='{self.nama}', lat={self.latitude:.4f}, lon={self.longitude:.4f})"

    def __str__(self) -> str:
        return f"{self.nama} [{type(self).__name__}]"

class TempatWisata(Lokasi):
    def __init__(self, nama: str, latitude: float, longitude: float, jenis: str, deskripsi: str):
        super().__init__(nama, latitude, longitude)
        self.jenis_wisata = str(jenis) if jenis else "Umum"
        self.deskripsi = str(deskripsi) if deskripsi else "Tidak ada deskripsi."

    def get_info_popup(self) -> str:
        return f"<h4><b>{self.nama}</b></h4><i>{self.jenis_wisata}</i><br><br>{self.deskripsi}<br><br>Koordinat: ({self.latitude:.4f}, {self.longitude:.4f})"

class Kuliner(Lokasi):
    def __init__(self, nama: str, latitude: float, longitude: float, menu_andalan: str):
        super().__init__(nama, latitude, longitude)
        self.menu_andalan = str(menu_andalan) if menu_andalan else "Tidak diketahui"

    def get_info_popup(self) -> str:
        return f"<h4><b>{self.nama}</b></h4><i>Kuliner</i><br><br>Menu Andalan: {self.menu_andalan}<br><br>Koordinat: ({self.latitude:.4f}, {self.longitude:.4f})"

class TempatIbadah(Lokasi):
    def __init__(self, nama: str, latitude: float, longitude: float, agama: str = "Umum", deskripsi: str = ""):
        super().__init__(nama, latitude, longitude)
        self.agama = str(agama) if agama else "Umum"
        self.deskripsi = str(deskripsi) if deskripsi else "Tempat Ibadah"

    def get_info_popup(self) -> str:
        return f"<h4><b>{self.nama}</b></h4><i>Tempat Ibadah ({self.agama})</i><br><br>{self.deskripsi}<br><br>Koordinat: ({self.latitude:.4f}, {self.longitude:.4f})"

class Museum(Lokasi):
    def __init__(self, nama: str, latitude: float, longitude: float, deskripsi: str = "Museum umum."):
        super().__init__(nama, latitude, longitude)
        self.deskripsi = deskripsi

    def get_info_popup(self) -> str:
        return f"<h4><b>{self.nama}</b></h4><i>Museum</i><br><br>{self.deskripsi}<br><br>Koordinat: ({self.latitude:.4f}, {self.longitude:.4f})"


class Kantor(Lokasi):
    def __init__(self, nama: str, latitude: float, longitude: float, deskripsi: str = "Gedung pemerintahan."):
        super().__init__(nama, latitude, longitude)
        self.deskripsi = deskripsi

    def get_info_popup(self) -> str:
        return f"<h4><b>{self.nama}</b></h4><i>Kantor Pemerintahan</i><br><br>{self.deskripsi}<br><br>Koordinat: ({self.latitude:.4f}, {self.longitude:.4f})"


class Taman(Lokasi):
    def __init__(self, nama: str, latitude: float, longitude: float, deskripsi: str = "Taman kota."):
        super().__init__(nama, latitude, longitude)
        self.deskripsi = deskripsi

    def get_info_popup(self) -> str:
        return f"<h4><b>{self.nama}</b></h4><i>Taman Kota</i><br><br>{self.deskripsi}<br><br>Koordinat: ({self.latitude:.4f}, {self.longitude:.4f})"


# --- Fungsi baca data (Salin dari Praktikum 2) ---
def baca_data_lokasi(nama_file: str) -> pd.DataFrame | None:
    # print(f"Mencoba membaca file CSV: {nama_file}") # Kurangi verbosity
    try:
        dataframe = pd.read_csv(nama_file)
        return dataframe
    except FileNotFoundError:
        print(f"ERROR: File '{nama_file}' tidak ditemukan!")
        return None
    except Exception as e:
        print(f"ERROR saat membaca file CSV: {type(e).__name__} - {e}")
        return None

# --- Fungsi Inti Praktikum Ini ---
def buat_objek_lokasi_dari_df(dataframe: pd.DataFrame) -> list:
    """
    Mengiterasi DataFrame Pandas dan membuat list berisi objek-objek
    Lokasi (atau turunannya) berdasarkan data di setiap baris.

    Args:
        dataframe (pd.DataFrame): DataFrame yang berisi data lokasi
                                  dengan kolom 'Nama', 'Latitude', 'Longitude',
                                  'Tipe', 'Deskripsi'.

    Returns:
        list: List berisi instance objek Lokasi atau turunannya.
    """
    list_objek_lokasi = []
    if dataframe is None or dataframe.empty:
        print("DataFrame kosong atau None, tidak ada objek dibuat.")
        return list_objek_lokasi

    print("\nMembuat objek dari DataFrame...")
    # Iterasi setiap baris dalam DataFrame
    for index, row in dataframe.iterrows():
        # Ambil data dari setiap kolom di baris saat ini
        # Gunakan .get() dengan default value untuk antisipasi kolom hilang
        nama = row.get('Nama', None)
        lat = row.get('Latitude', None)
        lon = row.get('Longitude', None)
        tipe = row.get('Tipe', 'Lainnya')  # Default jika kolom Tipe tidak ada
        deskripsi = row.get('Deskripsi', '')  # Default string kosong

        objek = None
        # Lakukan pengecekan tipe data dasar sebelum membuat objek
        if nama is None or lat is None or lon is None:
            print(f" -> Melewati baris {index}: Data Nama/Latitude/Longitude tidak lengkap.")
            continue  # Lanjut ke baris berikutnya

        # Membuat objek berdasarkan nilai di kolom 'Tipe'
        try:
            if 'Wisata' in tipe or tipe == 'Landmark':
                objek = TempatWisata(nama, lat, lon, tipe, deskripsi)
            elif tipe == 'Kuliner':
                objek = Kuliner(nama, lat, lon, deskripsi)
            elif 'Ibadah' in tipe:
                agama_info = "Umum"
                if "Islam" in tipe:
                    agama_info = "Islam"
                elif "Kristen" in tipe:
                    agama_info = "Kristen"
                elif "Klenteng" in tipe:
                    agama_info = "Tridharma"
                objek = TempatIbadah(nama, lat, lon, agama_info, deskripsi)
            elif tipe == 'Museum':
                objek = Museum(nama, lat, lon, deskripsi)
            elif tipe == 'Kantor Pemerintahan':
                objek = Kantor(nama, lat, lon, deskripsi)
            elif tipe == 'Taman Kota':
                objek = Taman(nama, lat, lon, deskripsi)
            else:
                print(f" -> Peringatan: Tipe '{tipe}' untuk '{nama}' tidak dikenali. Tidak membuat objek spesifik.")

                # Karena Lokasi abstrak, kita tidak bisa membuat objek Lokasi generik.
                # Jika Lokasi tidak abstrak, bisa: objek = Lokasi(nama, lat, lon)

            if objek:
                list_objek_lokasi.append(objek)
                # print(f" -> Objek {type(objek).__name__} untuk '{nama}' dibuat.")

        except Exception as e:
            # Tangani error saat pembuatan objek (misal konversi tipe gagal di __init__)
            print(f" -> GAGAL membuat objek untuk '{nama}' di baris {index}: {e}")

    print(f"Total {len(list_objek_lokasi)} objek lokasi berhasil dibuat dari {len(dataframe)} baris data.")
    return list_objek_lokasi

# --- Kode Utama ---
if __name__ == "__main__":
    NAMA_FILE_CSV = "lokasi_semarang.csv"

    print("--- Memulai Praktikum 4: Membuat Objek dari Data Pandas ---")
    # 1. Baca data CSV
    df_lokasi = baca_data_lokasi(NAMA_FILE_CSV)

    # 2. Buat list objek dari DataFrame
    list_semua_lokasi = buat_objek_lokasi_dari_df(df_lokasi)

    # 3. Tampilkan hasil (representasi objek)
    print("\n--- Daftar Objek Lokasi yang Berhasil Dibuat ---")
    if list_semua_lokasi:
        for idx, lok in enumerate(list_semua_lokasi):
            # repr() akan menunjukkan tipe objeknya (TempatWisata, Kuliner, dll.)
            print(f"{idx+1}. {repr(lok)}")
    else:
        print("Tidak ada objek lokasi yang dibuat.")

    print("\n--- Praktikum 4 Selesai ---")

--- Memulai Praktikum 4: Membuat Objek dari Data Pandas ---

Membuat objek dari DataFrame...
Total 13 objek lokasi berhasil dibuat dari 13 baris data.

--- Daftar Objek Lokasi yang Berhasil Dibuat ---
1. TempatWisata(nama='Lawang Sewu', lat=-6.9840, lon=110.4105)
2. TempatWisata(nama='Simpang Lima', lat=-6.9929, lon=110.4200)
3. TempatIbadah(nama='Masjid Agung Jawa Tengah', lat=-6.9892, lon=110.4452)
4. TempatIbadah(nama='Klenteng Sam Poo Kong', lat=-6.9980, lon=110.4030)
5. TempatWisata(nama='Brown Canyon', lat=-7.0375, lon=110.4875)
6. Kuliner(nama='Lumpia Gang Lombok', lat=-6.9718, lon=110.4255)
7. TempatWisata(nama='Kota Lama', lat=-6.9690, lon=110.4250)
8. TempatWisata(nama='Pantai Marina', lat=-6.9585, lon=110.3875)
9. TempatWisata(nama='Kampoeng Kopi Banaran', lat=-7.2780, lon=110.4010)
10. Kuliner(nama='Toko Oen', lat=-6.9715, lon=110.4235)
11. Museum(nama='Museum Ronggowarsito', lat=-6.9931, lon=110.3752)
12. Taman(nama='Taman Indonesia Kaya', lat=-6.9915, lon=110.4211)
13. Ka